In [ ]:
import os
import re
import csv
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
import subprocess
from subprocess import check_output
import matplotlib.pyplot as plt
import pandas as pd
import time
import hashlib
import numpy as np

df = pd.read_csv("single_fio.csv")

def extract_values(input_list):
    numeric_values = []
    converted_values = []

    for value_with_unit in input_list:
        match = re.match(r'(\d+\.\d+|\d+)(\w+)/s', value_with_unit)
        if match:
            numeric_value = float(match.group(1))
            unit = match.group(2)

            if unit == 'KiB':
                numeric_value /= 1024  # Convert KiB to MiB

            numeric_values.append(numeric_value)
            converted_values.append(f'{numeric_value}MiB/s')

    return numeric_values

def condition_generator(bs, qd, cs, pt, dt):
    conditions_lsvd_ssd_rr = (
        (df['pool_type'] == pt) & 
        (df['disk_type'] == dt) & 
        (df['cache_size'] == cs) & 
        (df['workload'] == 'randread') &
        (df['bs'] == bs) &
        (df['iodepth']==qd)
    )
    
    conditions_lsvd_ssd_rw = (
        (df['pool_type'] == pt) & 
        (df['disk_type'] == dt) & 
        (df['cache_size'] == cs) & 
        (df['workload'] == 'randwrite') &
        (df['bs'] == bs) &
        (df['iodepth']==qd)
    )
    
    conditions_lsvd_ssd_sr = (
        (df['pool_type'] == pt) & 
        (df['disk_type'] == dt) & 
        (df['cache_size'] == cs) & 
        (df['workload'] == 'read') &
        (df['bs'] == bs) &
        (df['iodepth']==qd)
    )
    
    conditions_lsvd_ssd_sw = (
        (df['pool_type'] == pt) & 
        (df['disk_type'] == dt) & 
        (df['cache_size'] == cs) & 
        (df['workload'] == 'write') &
        (df['bs'] == bs) &
        (df['iodepth']==qd)
    )
    return conditions_lsvd_ssd_rr, conditions_lsvd_ssd_rw, conditions_lsvd_ssd_sr, conditions_lsvd_ssd_sw 

def lsvd_smallc_ssd:
    bs='4ki'
    qd=256
    cs='20gb'
    pt='ssd'
    dt='lsvd'
    
    conditions_20gb_lsvd_ssd_rr, conditions_20gb_lsvd_ssd_rw, conditions_20gb_lsvd_ssd_sr, conditions_20gb_lsvd_ssd_sw = condition_generator(bs, qd, cs, pt, dt)
    
    pt ='hdd'
    
    conditions_20gb_lsvd_hdd_rr, conditions_20gb_lsvd_hdd_rw, conditions_20gb_lsvd_hdd_sr, conditions_20gb_lsvd_hdd_sw = condition_generator(bs, qd, cs, pt, dt)
    
    dt='rbd'
    pt='ssd'
    cs='none'
    
    conditions_rbd_ssd_rr, conditions_rbd_ssd_rw, conditions_rbd_ssd_sr, conditions_rbd_ssd_sw = condition_generator(bs, qd, cs, pt, dt)
    #condition for rbd_ssd
    
    
    pt='hdd'
    
    conditions_rbd_hdd_rr, conditions_rbd_hdd_rw, conditions_rbd_hdd_sr, conditions_rbd_hdd_sw = condition_generator(bs, qd, cs, pt, dt)
    
    bw_20gb_lsvd_ssd_rr=df[conditions_20gb_lsvd_ssd_rr]
    bw_20gb_lsvd_ssd_rr=bw_20gb_lsvd_ssd_rr['bw'].iloc[0]
    # print(bw_20gb_lsvd_ssd_rr)
    
    bw_rbd_ssd_rr=df[conditions_rbd_ssd_rr]
    bw_rbd_ssd_rr=bw_rbd_ssd_rr['bw'].iloc[0]
    # print(bw_rbd_ssd_rr)
    
    bw_20gb_lsvd_ssd_rw=df[conditions_20gb_lsvd_ssd_rw]
    bw_20gb_lsvd_ssd_rw=bw_20gb_lsvd_ssd_rw['bw'].iloc[0]
    # print(bw_20gb_lsvd_ssd_rw)
    
    bw_rbd_ssd_rw=df[conditions_rbd_ssd_rw]
    bw_rbd_ssd_rw=bw_rbd_ssd_rw['bw'].iloc[0]
    #print(bw_rbd_ssd_rw)
    
    bw_20gb_lsvd_ssd_sr=df[conditions_20gb_lsvd_ssd_sr]
    bw_20gb_lsvd_ssd_sr=bw_20gb_lsvd_ssd_sr['bw'].iloc[0]
    #print(bw_20gb_lsvd_ssd_sr)
    
    bw_rbd_ssd_sr=df[conditions_rbd_ssd_sr]
    bw_rbd_ssd_sr=bw_rbd_ssd_sr['bw'].iloc[0]
    #print(bw_rbd_ssd_sr)
    
    bw_20gb_lsvd_ssd_sw=df[conditions_20gb_lsvd_ssd_sw]
    bw_20gb_lsvd_ssd_sw=bw_20gb_lsvd_ssd_sw['bw'].iloc[0]
    #print(bw_20gb_lsvd_ssd_sw)
    
    bw_rbd_ssd_sw=df[conditions_rbd_ssd_sw]
    bw_rbd_ssd_sw=bw_rbd_ssd_sw['bw'].iloc[0]
    #print(bw_rbd_ssd_sw)
    
    